- 현재 canny edge signal에 conditioning을 하는 것은 잘 되어 있다.
- 따라서, normal이나 depth에 대해서도 conditioning을 하고자 한다.

In [1]:
%load_ext autoreload
%autoreload 2

In [30]:
import os
import torch
from transformers import AutoTokenizer, AutoModel
import torchvision.transforms as T
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
import pandas as pd
from sklearn.model_selection import train_test_split


import numpy as np


In [4]:
# text dataset 가져오기
text_path = "/workspace/code/3DAnything/debug/data/captions.csv"

In [5]:
loaded_df = pd.read_csv(text_path)

In [7]:
loaded_df

,meta_class,class,dataset_type,base_name,caption
0,n02988304,dcb921b4517a45db8853bfdd4d0cafdc,image_render,000.png,The image shows a black external CD-ROM drive ...
1,n02988304,dcb921b4517a45db8853bfdd4d0cafdc,image_render,001.png,"The image shows a black, cube-shaped electroni..."
2,n02988304,dcb921b4517a45db8853bfdd4d0cafdc,image_render,002.png,The image shows a black stylus or pen-like dev...
3,n02988304,dcb921b4517a45db8853bfdd4d0cafdc,image_render,003.png,The image shows a black projector with a circu...
4,n02988304,dcb921b4517a45db8853bfdd4d0cafdc,image_render,004.png,The image shows a black Apple PowerCD external...
...,...,...,...,...,...
45389,n02441942,96f9026ddace492b8a3945c1f7c7d35d,image_render,010.png,"The image is a simple, stylized, gray 3D model..."
45390,n02441942,96f9026ddace492b8a3945c1f7c7d35d,image_render,011.png,The image shows a gray silhouette of a worm ag...
45391,n02441942,96f9026ddace492b8a3945c1f7c7d35d,image_render,012.png,The image shows a 3D model of a low-poly antea...
45392,n02441942,96f9026ddace492b8a3945c1f7c7d35d,image_render,013.png,"The image is a simple, gray 3D model of an ani..."


In [9]:
test_string = "n01440764/09f07baa8fa9496994aef0693ac7bec8/image_render/000.png"
test_string_split = test_string.split("/")

In [ ]:
loaded_df[
    (loaded_df["base_name"] == test_string_split[-1]) & 
    (loaded_df["class"] == test_string_split[-3])
]["caption"].values[0]

'The image shows a silver-colored fish with a distinctive hump on its head, likely a Batfish, against a black background. The fish has long, flowing fins and a unique body shape. There is a small "Fish Puzzle" logo in the bottom left corner.'

# dataset 정의


In [28]:
df = loaded_df.copy()
df['combined'] = df['meta_class'].astype(str) + '/' + df['class'].astype(str)
dataset_arr = df['combined'].unique()

In [31]:
train_arr, val_arr = train_test_split(dataset_arr, test_size=0.1, random_state=42)
# Train 파일 저장
with open('train.txt', 'w') as f:
    for item in train_arr:
        f.write("%s\n" % item)

# Validation 파일 저장
with open('val.txt', 'w') as f:
    for item in val_arr:
        f.write("%s\n" % item)

In [27]:
df["combined"].unique()

array(['n02988304/dcb921b4517a45db8853bfdd4d0cafdc',
       'n02988304/a4bc4accb20b43ffa728a7a85b7dfed7',
       'n02988304/7e12ac07201647debdada98877db130a', ...,
       'n02441942/8aa30bf875a0440cb763914baa8659b6',
       'n02441942/dac05797fdd74dd482b024595ff35d03',
       'n02441942/96f9026ddace492b8a3945c1f7c7d35d'], dtype=object)